In [1]:
import pandas as pd
import numpy as np

# from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
# from sklearn.ensemble import RandomForestClassifier, VotingClassifier,AdaBoostClassifier,GradientBoostingClassifier, BaggingClassifier
# from sklearn.metrics import classification_report
# from sklearn import preprocessing, neighbors, svm
# from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
# from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split, KFold

In [2]:
df_train = pd.read_csv('../data/train.csv')
df_test = pd.read_csv('../data/test.csv')

In [3]:
def get_data(df,label, test_size = 0.2):
  X = df.drop([label], axis=1)
  X = np.matrix(X)

  y = np.array(df[label])
  return train_test_split(X,y,test_size=test_size)

def validate(clf,df, test_size=0.2): 
  X_train, X_test, y_train, y_test = get_data(df,'label', test_size=test_size)
  clf.fit(X_train,y_train)
        
  y_pred = clf.predict(X_test)
  score = clf.score(X_test,y_test)
  return score


In [17]:
# clf = RandomForestClassifier(n_estimators=5000, n_jobs = -1, criterion='entropy') 
clf = RandomForestClassifier(n_estimators=1500, n_jobs = -1)
print(validate(clf,df_train))

0.967619047619


In [19]:
df_train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
clf = RandomForestClassifier(n_estimators=1500, n_jobs = -1, criterion='entropy') 
print(validate(clf,df_train))

0.964047619048


In [18]:
X_train = df_train.drop(['label'], axis=1)
X_train = np.matrix(X_train)

y_train = np.array(df_train.label)

X_test = np.matrix(df_test)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

y_pred = pd.DataFrame(y_pred)

y_pred.index += 1
y_pred.to_csv('../mnist_pred_1.csv', header=['Label'], index_label='ImageId')

In [157]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [318]:
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

In [319]:
y = tf.nn.softmax(tf.matmul(x,W) + b)

In [320]:
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

In [321]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [373]:
X_train = df_train.drop(['label'], axis=1)

X_train = np.divide(X_train, 255.0)
X_train = np.matrix(X_train)
print(X_train[0].shape)

X_test = np.matrix(df_test)
X_test = np.divide(X_test, 255.0)

y_train = np.array(df_train.label)
y_train_one_hot = np.zeros((len(y_train), 10))
y_train_one_hot[np.arange(len(y_train)), y_train] = 1

# max(mnist.train.images[0])
np.max(np.array(X_train[0]))

(1, 784)


1.0

In [323]:
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init)
# sess.run(train_step, feed_dict={x:X_train[:40000], y_:y_train_one_hot[:40000]})

In [324]:
for i in range(1000):
  batch_xs, batch_ys = X_train[i*40:(i+1)*40], y_train_one_hot[i*40:(i+1)*40]
  
  sess.run(train_step, feed_dict={x:batch_xs, y_:batch_ys})

In [325]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: X_train[40000:], y_:y_train_one_hot[40000:]}))

0.9055


In [372]:
sess.close()
print(X_test.shape)

(28000, 784)


In [374]:
sess = tf.InteractiveSession()

In [375]:
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

sess.run(tf.global_variables_initializer())

In [376]:
y = tf.nn.softmax(tf.matmul(x,W) + b)

In [377]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

In [378]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [263]:
for i in range(1000):
  batch_xs, batch_ys = X_train[i*40:(i+1)*40], y_train_one_hot[i*40:(i+1)*40]
  
  train_step.run(feed_dict={x: batch_xs, y_: batch_ys})

In [264]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={x: X_train[40000:], y_:y_train_one_hot[40000:]}))

0.9055


In [379]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [380]:
def conv2d(x,W):
  return tf.nn.conv2d(x, W, strides = [1,1,1,1], padding = 'SAME')
def max_pool_2x2(x):
  return tf.nn.max_pool(x,ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [381]:
W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])

In [382]:
x_image = tf.reshape(x,[-1,28,28,1])

In [383]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [384]:
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [385]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [386]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [387]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
y =tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [401]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())
for i in range(840):
  batch_xs, batch_ys = X_train[i*50:(i+1)*50], y_train_one_hot[i*50:(i+1)*50]
  
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch_xs, y_: batch_ys, keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 0.5})

print("test accuracy %g"%sess.run(accuracy,feed_dict={
    x: X_train[40000:], y_: y_train_one_hot[40000:], keep_prob: 1.0}))

# y_pred = sess.run(tf.argmax(y,1), {x:np.array(X_test), keep_prob: 1.0})

step 0, training accuracy 0.16
step 100, training accuracy 0.84
step 200, training accuracy 0.9
step 300, training accuracy 0.92
step 400, training accuracy 0.96
step 500, training accuracy 0.96
step 600, training accuracy 0.98
step 700, training accuracy 0.94
step 800, training accuracy 0.88
test accuracy 0.964


In [398]:
y_pred = []
for i in range(0,28000,7000):
  y_pred = y_pred + list(sess.run(tf.argmax(y,1), {x:np.array(X_test[i:i + 7000]), keep_prob: 1.0}))

print(len(y_pred))

28000


In [ ]:
# y_pred = sess.run(tf.argmax(y_conv,1), {x:X_test})
print(y_pred)

In [399]:
y_pred = pd.DataFrame(y_pred)

y_pred.index += 1
y_pred.to_csv('../mnist_pred_neural_1.csv', header=['Label'], index_label='ImageId')

In [400]:
print(y_pred.head())

   0
1  2
2  0
3  9
4  9
5  3
